<a href="https://colab.research.google.com/github/dzanahmed/welcome-ideathon-lshtm/blob/main/code/no_vaccination_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## Load Packages

In [26]:
import pandas as pd
import re
import tqdm
import nltk
import string

## Load data in dataframe

In [27]:
url = "https://raw.githubusercontent.com/dzanahmed/welcome-ideathon-lshtm/main/data/interim/vax_tweets_v0_filtered_unfiltered.csv?token=GHSAT0AAAAAACB5CGERHJBFG744WYKK3L3GZFJPQUQ"
raw_df = pd.read_csv(url)

# raw_df.dtypes

<ipython-input-27-6450e61c635d>:2: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(url)


## save necessary columns in new dataframe

In [28]:
df = raw_df[['tweet_id', 'text', 'filtered']]
df['text'] = df['text'].astype(str)

# df.head()

<ipython-input-28-f24e558cbac7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)


## Regular Expressions to Capture Posts Describing Decision to Decline the COVID-19 Vaccine

In [29]:
# REGULAR EXPRESSIONS TAKEN DIRECTLY FROM:
# Golder S, McRobbie-Johnson ACE, Klein A, Polite FG, Gonzalez Hernandez G. Social media and COVID-19 vaccination hesitancy during pregnancy: a mixed methods analysis. BJOG. 2023;130(6):750–758. https://doi.org/10.1111/1471- 0528.17481

REGEXES_NO_COVID_VACCINE_UPDATED = [
r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(\S+\s*)?(will\s*not|won\W*t|would\s*not|wouldn\W*t)\s*(get|have)\s*((the|a)\s*)?#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnik\W*v|mrna|\W)*(vaccines?|vaccinations?|vaxx?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(\S+\s*)?(will\s*not|won\W*t|would\s*not|wouldn\W*t)\s*(get|have)\s*(the|a)\s+#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnik\W*v|mrna|\W)+(shots?|jabs?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi(\s*am|\W*m|)\s*(\S+\s*)?not\s*(getting|having)\s*((the|a)\s*)?#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnikv|sputnik\W*v|mrna|\W)*(vaccines?|vaccinations?|vaxx?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi(\s*am|\W*m|)\s*(\S+\s*)?not\s*(getting|having)\s*(the|a)\s+#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnikv|sputnik\W*v|mrna|\W)+(shots?|jabs?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(\S+\s*)?(will\s*not|won\W*t)\s*be\s*(getting|having)\s*((the|a)\s*)?#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnikv|sputnik\W*v|mrna|\W)*(vaccines?|vaccinations?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(\S+\s*)?(will\s*not|won\W*t)\s*be\s*(getting|having)\s*(the|a)\s+#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnikv|sputnik\W*v|mrna|\W)+(shots?|jabs?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\b(i|i\s*am|i\W*m|i\s*have|i\W*ve)\s*(\S+\s*)?(refus(e|ed|ing)|oppos(e|ed|ing)|reject(ed|ing)?|object(ed|ing)?)\s*(to\s*)?((get|getting|having)\s*)?((the|a)\s*)?#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnikv|sputnik\W*v|mrna|\W)*(vaccines?|vaccinations?|vaxx?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\b(i|i\s*am|i\W*m|i\s*have|i\W*ve)\s*(\S+\s*)?(refus(e|ed|ing)|oppos(e|ed|ing)|reject(ed|ing)?|object(ed|ing)?)\s*(to\s*)?((get|getting|have|having)\s*)?(the|a)\s+#?(covid|covid\W*19|coronavirus|corona|moderna|pfizer|astrazeneca|biontech|janssen|johnson\W*johnson|johnson&amp;johnson|j\W*j|j&amp;j|novavax|sputnikv|sputnik\W*v|mrna|\W)+(shots?|jabs?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(\S+\s*)?(will\s*not|won\W*t)\s*(get|be)\s*(vaccinated|vaxed|vaxxed|jabbed)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi(\s*am|\W*m)\s*(\S+\s*)?not\s*((getting|being)\s*)?(vaccinated|vaxed|vaxxed|jabbed)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi(\s*have|\W*ve)\s*not\s*been\s*(vaccinated|vaxed|vaxxed|jabbed)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\b(i|i\s*am|i\W*m|i\s*have|i\W*ve)\s*(\S+\s*)?(refus(e|ed|ing)|oppos(e|ed|ing)|object(ed|ing)?)\s*(to\s*)?(get|getting|be|being)\s*(vaccinated|vaxed|vaxxed|jabbed)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\b(not|haven\W*t|didn\W*t)\s*(get|got|gotten|have|had|received)\s*my\s*#?((covid|covid\W*19|coronavirus|corona)\s*)?(vaccines?|vaccinations?|vaxx?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\b(not|haven\W*t|didn\W*t)\s*(get|got|gotten|have|had|received)\s*my\s*#?(covid|covid\W*19|coronavirus|corona)\s*(shots?|jabs?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(have\s*not|haven\W*t|did\s*not|didn\W*t)\s*(get|got|gotten|have|had|received)\s*(the|a)\s*#?((covid|covid\W*19|coronavirus|corona)\s*)?(vaccines?|vaccinations?|vaxx?)\b',

r'(?<!“)(?<!")(?<!‘)(?<!:)(?<!“\s)(?<!"\s)(?<!‘\s)(?<!:\s)\bi\s*(have\s*not|haven\W*t|did\s*not|didn\W*t)\s*(get|got|gotten|have|had|received)\s*(the|a)\s*#?(covid|covid\W*19|coronavirus|corona)\s*(shots?|jabs?)\b',

]

## Preprocess tweets -- lowercase, remove punctuation

In [30]:
# Define function to preprocess a tweet
def preprocess(tweet):
  tweet=str(tweet)
  tweet = tweet.lower()
  punctuation = string.punctuation
  tweet = ''.join([char for char in tweet if char not in punctuation])

  # sentence=sentence.replace('{html}',"")
  # cleanr = re.compile('<.*?>')
  # cleantext = re.sub(cleanr, '', sentence)
  # rem_url=re.sub(r'http\S+', '', tweet)
  # rem_num = re.sub('[0-9]+', '', rem_url)
  # tokenizer = RegexpTokenizer(r'\w+')
  # tokens = tokenizer.tokenize(rem_num)
  # filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  # stem_words=[stemmer.stem(w) for w in filtered_words]
  # lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  # return " ".join(filtered_words)
  return tweet

In [31]:
# preprocess every tweet in the dataframe and save it in a new column, clean_text
df['clean_text'] = df['text'].map(lambda tweet :preprocess(tweet))

<ipython-input-31-b875e50a7d52>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['text'].map(lambda tweet :preprocess(tweet))


# Filter through tweets

In [32]:
# Create an empty boolean column 'no_vax_filter' in the DataFrame
df['no_vax_filter'] = False

# Iterate over each regex pattern
for pattern in REGEXES_NO_COVID_VACCINE_UPDATED:
    # Use 'str.contains()' to check if each tweet matches the pattern
    mask = df['clean_text'].str.contains(pattern, flags=re.IGNORECASE, regex=True)

    # Update 'no_vax_filter' column where the tweet matches the pattern
    df.loc[mask, 'no_vax_filter'] = True

# Print the DataFrame with the 'no_vax_filter' column
print(df.head())

<ipython-input-32-8cdaf0f0b726>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['no_vax_filter'] = False
<ipython-input-32-8cdaf0f0b726>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['clean_text'].str.contains(pattern, flags=re.IGNORECASE, regex=True)


   tweet_id                                               text  filtered  \
0         1  We asked our coworkers why they're getting a C...      True   
1         2  45+ #RURAL #Bengaluru #CovidVaccine Availabili...      True   
2         3  @JoyAnnReid @NIH  👿Questions:  Could the vacci...      True   
3         4  Next question is how do you find out where you...      True   
4         5  If you told your child to get a Covid vaccine,...      True   

                                          clean_text  no_vax_filter  
0  we asked our coworkers why theyre getting a co...          False  
1  45 rural bengaluru covidvaccine availability f...          False  
2  joyannreid nih  👿questions  could the vaccine ...          False  
3  next question is how do you find out where you...          False  
4  if you told your child to get a covid vaccine ...          False  


## Examine regex filtering outputs

In [33]:
# report number of positive/negative classified tweets
print(f"Passed filters: {len(df[df['no_vax_filter'] == True])}")
print(f"Did not pass filters: {len(df[df['no_vax_filter'] == False])}")

# inspect classification of tweets
print("\nTweets that passed the filters:")
df.loc[df['no_vax_filter'] == True, ['text', 'no_vax_filter']]

Passed filters: 60
Did not pass filters: 99937

Tweets that passed the filters:


,text,no_vax_filter
2768,@realDonaldTrump Well then stop with the vacci...,True
6407,A concise summary of the #CovidVaccine rollout...,True
6626,People over 60 who've had their vaccines walki...,True
6656,So a member of my family today posted a anti c...,True
8047,@MarkinTenn This is WHY I won't get the #Covid...,True
8573,#CovidVaccine I recommend not getting the vacc...,True
11826,@TorahLectures I didn't get the #CovidVaccine ...,True
12194,@AP So... a booster shot. Glad I didn't get m...,True
14949,@leeshutterbug @dianemariereid Same age so I w...,True
18702,@offspring You're not PUNK! You're Govt. LACKE...,True


In [34]:
from google.colab import files

output_df = df.loc[:, ['tweet_id', 'no_vax_filter']]
output_df.to_csv('no_vax_filter_results.csv', encoding = 'utf-8-sig')
files.download('no_vax_filter_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>